In [16]:
import numpy as np
import sklearn.preprocessing
import torch
from sklearn.model_selection import train_test_split
import torch
from matplotlib import pyplot as plt 
import random
import torch.nn as nn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import Dataset, DataLoader, TensorDataset, Subset, SubsetRandomSampler
import os
import json
import torchvision 
import torchvision.transforms as transforms
from torch.utils.data import random_split
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm
from sklearn.model_selection import KFold
import pandas as pd
from time import time
from torchvision.datasets import ImageFolder
from torch import as_strided


device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [34]:
train_ds = np.load('./NYC-stdn/volume_train.npz')['volume']
test_ds = np.load('./NYC-stdn/volume_test.npz')['volume']

In [35]:
print(train_ds.shape)
print(test_ds.shape)

(1920, 10, 20, 2)
(960, 10, 20, 2)


In [36]:
train_ds = train_ds.reshape(train_ds.shape[0], -1, train_ds.shape[3])
test_ds = test_ds.reshape(test_ds.shape[0], -1, test_ds.shape[3])
print(train_ds.shape)
print(test_ds.shape)

(1920, 200, 2)
(960, 200, 2)


In [38]:
train_scaler = np.zeros(train_ds.shape)
test_scaler = np.zeros(test_ds.shape)
i_train_scaler = MinMaxScaler()  # 入的流量
o_train_scaler = MinMaxScaler()  # 出的流量
i_test_scaler = MinMaxScaler()  
o_test_scaler = MinMaxScaler()
# fit只支持2维的操作
train_scaler[:,:,0] = i_train_scaler.fit_transform(train_ds[:,:,0])
train_scaler[:,:,1] = o_train_scaler.fit_transform(train_ds[:,:,1])

test_scaler[:,:,0] = i_test_scaler.fit_transform(test_ds[:,:,0])
test_scaler[:,:,1] = o_test_scaler.fit_transform(test_ds[:,:,1])

# 划分训练数据集，验证数据集，测试数据集
train_proportion, val_proportion = 0.8, 0.2
train_dataset = train_scaler[:int(train_proportion * len(train_ds)),:,:]
val_dataset = train_scaler[int(train_proportion * len(train_ds)):,:,:]

# train_dl = DataLoader(train_dataset, batch_size=64, shuffle=False)
# val_dl = DataLoader(val_dataset, batch_size=64, shuffle=False)
# test_dl = DataLoader(test_ds, batch_size=64, shuffle=False)

In [39]:
class TrafficDataset(Dataset):
    def __init__(self, dataset, window_size, pred_len):
        self.window_size = window_size
        self.pred_len = pred_len
        self.dataset = dataset
        self.T, self.N, self.F = self.dataset.shape  # 时间步，节点数，特征数
    
    def __len__(self):
        return self.T - self.window_size - self.pred_len + 1
    
    def __getitem__(self, idx):
        x = self.dataset[idx:idx + self.window_size, :, :]
        y = self.dataset[idx + self.window_size:idx + self.window_size + self.pred_len, :, :]
        x = torch.FloatTensor(x)  # shape: (window_size, N, F)
        y = torch.FloatTensor(y)  # shape: (pred_len, N, F)
        x = x.view(self.window_size, -1)
        y = y.view(self.pred_len, -1)
        return x, y

In [40]:
train_ds = TrafficDataset(train_ds, window_size=6, pred_len=1)
test_ds = TrafficDataset(test_ds, window_size=6, pred_len=1)
train_dl = DataLoader(train_ds, batch_size=32, shuffle=False)
test_dl = DataLoader(test_ds, batch_size=32, shuffle=False)

In [42]:
for i, (x, y) in enumerate(train_dl):
    print(x.shape)
    print(y.shape)
    break

torch.Size([32, 6, 400])
torch.Size([32, 1, 400])
